In [11]:
import requests
import pandas as pd
import json
import mysql.connector
from datetime import datetime, timedelta
import time
import nvda_config2

import twelvedata
from twelvedata import TDClient



In [12]:
# set up login
Cpassword = nvda_config2.mysql_password
td = TDClient(apikey=nvda_config2.twelve_api_nvda) 
base_url = 'https://api.twelvedata.com'

# Establish MySQL object
mydb = mysql.connector.connect(
    host="localhost",   
    user="root",         
    password=Cpassword,  
    database="nvdav1"   
)

mycursor = mydb.cursor()

# Twelve Data API parameters
symbol = "NVDA, SPCE, ABAT, BB, GME, GRRR, MAXN, TOMZ"
interval = "1min"
outputsize = 5000  

In [13]:
end_date = datetime.now()
start_date = end_date - timedelta(days=1) 
time_batch = td.time_series(symbol=symbol, interval=interval, outputsize=outputsize, start_date=start_date.strftime("%Y-%m-%d"),
        end_date=end_date.strftime("%Y-%m-%d"))
df = time_batch.with_macd().with_macd(fast_period=10).with_stoch().as_pandas()




In [27]:
# Store current prices as old_prices
old_prices = price_df.copy() if 'price_df' in locals() else pd.DataFrame()

# Get new prices
price_batch = td.price(symbol=symbol)
price_json = price_batch.as_json()
price_df = pd.DataFrame(price_json, index=["price"])

# Calculate the difference

if not old_prices.empty:
    diff = price_df.astype(float) - old_prices.astype(float)
    
    # Create a DataFrame to store old price, difference, and new price
    result_df = pd.DataFrame({
        'Old Price': old_prices.iloc[0],
        'Difference': diff.iloc[0],
        'New Price': price_df.iloc[0]
    })
    
    # Display the results
    print("Price Changes:")
    print(result_df)
else:
    print("Initial prices:")
    print(price_df)

df.to_csv('time_data.csv', index=False)
price_df.to_csv('price_data.csv', index=False)

Price Changes:
      Old Price  Difference  New Price
NVDA  112.50000      0.5600  113.06000
SPCE    7.42500      0.0250    7.45000
ABAT    0.99510      0.0149    1.01000
BB      2.47500      0.0050    2.48000
GME    24.19850     -0.0685   24.13000
GRRR    2.87000      0.0000    2.87000
MAXN    0.21110      0.0019    0.21300
TOMZ    0.74500      0.0148    0.75980
